# triggering render jobs

Start by running $WORKSPACE/tools/fission/reset-fission-proxies before launching the notebook server.

#### (Re)deploy function and trigger

In [9]:
%%bash
sh deploy.sh

function 'job-trigger-render' deleted
trigger 'ebd9b767-1fc0-4428-b5cd-d8343019a6b5' deleted
function 'job-trigger-render' created
trigger '46ef36d4-12b0-412e-9456-9175b6619a42' created


#### Update and trigger function

In [64]:
%%bash
sh dev.sh

function 'job-trigger-render' updated
{'api_version': 'batch/v1',
 'kind': 'Job',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2018, 1, 22, 17, 40, 9, tzinfo=tzlocal()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'initializers': None,
              'labels': {'controller-uid': '4a8c23c4-ff9b-11e7-b604-42010af00218',
                         'job-name': 'pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc'},
              'name': 'pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc',
              'namespace': 'rl',
              'owner_references': None,
              'resource_version': '1780363',
              'self_link': '/apis/batch/v1/namespaces/rl/jobs/pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc',
              'uid': '4a8c23c4-ff9b-11e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6203  100  6113  100    90  19802    291 --:--:-- --:--:-- --:--:-- 19847


In [72]:
%%bash
kubectl get pods -n rl

NAME                                                    READY     STATUS    RESTARTS   AGE
pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc-vg8rc   1/1       Running   0          28s


In [95]:
%%bash
kubectl logs pybullet-kuka-ff-0118-2346-bac2-render-cf11f2dc-vg8rc -n rl

INFO:tensorflow:Resume run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2.
INFO:tensorflow:Resume run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2.
{'algorithm': <class 'agents.ppo.algorithm.PPOAlgorithm'>,
 'discount': 0.995,
 'env': 'KukaBulletEnv-v0',
 'eval_episodes': 25,
 'init_logstd': -1,
 'init_mean_factor': 0.1,
 'kl_cutoff_coef': 1000,
 'kl_cutoff_factor': 2,
 'kl_init_penalty': 1,
 'kl_target': 0.01,
 'learning_rate': 0.0001,
 'logdir': 'gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2',
 'max_length': 1000,
 'network': <function feed_forward_gaussian at 0x7f0afc908b18>,
 'num_agents': 30,
 'optimizer': <class 'tensorflow.python.training.adam.AdamOptimizer'>,
 'optimizer_pre_initialize': True,
 'policy_layers': (200, 100),
 'steps': 40000000.0,
 'update_epochs': 25,
 'update_every': 30,
 'use_gpu': False,
 'value_layers': (200, 100),
 'weight_summaries': {'all': '.*',


In [96]:
%%bash
gsutil ls gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render

gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.episode_batch.0.46.stats.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.manifest.0.46.manifest.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000000.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000000.mp4
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000001.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000001.mp4
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000002.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000002.mp4
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render/openaigym.video.0.46.video000003.meta.json
gs://kubeflow-rl-kf/jobs/pybullet-kuka-ff-0118-2346-bac2/render

Create a job on kubeflow showing display of job trigger request every N seconds (not submitted).

In [98]:
%%bash

get_project_id() {
  project=$(gcloud config get-value project 2> /dev/null)
  if [[ -z "$project" ]]; then
      >&2 echo "Couldn't load a gcloud project ID!"
  fi
  echo "$project"
}

cd ../../../rl-app


HPARAM_ID=kuka

JOB_SALT=`date | shasum -a 256 | cut -c1-8`
JOB_NAME=`echo ${HPARAM_ID}-${JOB_SALT} | tr '_' '-'`
LOG_DIR=gs://$(get_project_id)-kf/jobs/${JOB_NAME}

# 1.4.1, agents propper, MTS, periodic render messages
IMAGE=gcr.io/kubeflow-rl/agents-ppo:cpu-df1d364a

ks param set agents-ppo env "KukaBulletEnv-v0"

ks param set agents-ppo run_mode train
ks param set agents-ppo gcp_project kubeflow-rl
ks param set agents-ppo num_cpu 1
ks param set agents-ppo num_agents 1
ks param set agents-ppo sync_replicas False
ks param set agents-ppo steps 1000

# Trigger an async render job every 10 seconds (for demo)
ks param set agents-ppo render_secs 10

ks param set agents-ppo algorithm "agents.ppo.PPOAlgorithm"
ks param set agents-ppo network "agents.scripts.networks.feed_forward_gaussian"

ks param set agents-ppo job_tag ${JOB_SALT}
ks param set agents-ppo logdir ${LOG_DIR}
ks param set agents-ppo name ${JOB_NAME}
ks param set agents-ppo image ${IMAGE}

ks apply gke -c agents-ppo

Parameter 'env' successfully set to '"KukaBulletEnv-v0"' for component 'agents-ppo'
Parameter 'run_mode' successfully set to '"train"' for component 'agents-ppo'
Parameter 'gcp_project' successfully set to '"kubeflow-rl"' for component 'agents-ppo'
Parameter 'num_cpu' successfully set to '1' for component 'agents-ppo'
Parameter 'num_agents' successfully set to '1' for component 'agents-ppo'
Parameter 'sync_replicas' successfully set to '"False"' for component 'agents-ppo'
Parameter 'steps' successfully set to '1000' for component 'agents-ppo'
Parameter 'render_secs' successfully set to '10' for component 'agents-ppo'
Parameter 'algorithm' successfully set to '"agents.ppo.PPOAlgorithm"' for component 'agents-ppo'
Parameter 'network' successfully set to '"agents.scripts.networks.feed_forward_gaussian"' for component 'agents-ppo'
Parameter 'job_tag' successfully set to '"e456acf9"' for component 'agents-ppo'
Parameter 'logdir' successfully set to '"gs://kubeflow-rl-kf/jobs/kuka-e456acf9"'

In [103]:
%%bash
kubectl get pods -n rl

NAME                                READY     STATUS    RESTARTS   AGE
kuka-e456acf9-master-dsse-0-95xzd   1/1       Running   0          20s


Note that in the following elasped time increments by render_secs

In [117]:
%%bash
kubectl logs kuka-e456acf9-master-dsse-0-95xzd -n rl

INFO:tensorflow:Start a new run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/kuka-e456acf9.
2018-01-22 22:44:40.459919: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
E0122 22:44:40.550151431       1 ev_epoll1_linux.c:1051]     grpc epoll fd: 3
2018-01-22 22:44:40.558247: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:215] Initialize GrpcChannelCache for job master -> {0 -> localhost:2222}
2018-01-22 22:44:40.561891: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:324] Started server with target: grpc://localhost:2222
{'algorithm': <class 'agents.ppo.algorithm.PPOAlgorithm'>,
 'debug': True,
 'discount': 0.995,
 'dump_dependency_versions': False,
 'env': 'KukaBulletEnv-v0',
 'env_processes': True,
 'eval_episodes': 25,
 'hparam_set_id': 'pybullet_kuka_ff',
 'init_logstd': -1,
 'init_mean_factor': 0.1,
 'kl_cutoff_coef': 1000,
 '

In [63]:
%%bash
kubectl delete jobs --all -n rl 

job "pybullet-kuka-ff-0118-2346-bac2-render-69b136bb" deleted
job "pybullet-kuka-ff-0118-2346-bac2-render-9fe7548f" deleted


### Status:
- Currently finishes without error but renders do not arrive on cloud storage.
- Should be rendering to /tmp then uploading render when complete